In [1]:
import pandas as pd
import openpyxl
from pandas import DataFrame
from sklearn import tree
import pickle
import json

In [2]:
printscoreone=0
printscoretwo=0
name_list=[]
temp_name_list=[]

In [3]:
df = pd.read_csv('D:\\Cloud Counselage\\Data.csv')

In [4]:
dmUQ = df.Domain.unique()
dmNO=[]
[dmNO.append(x) for x in range(1,len(dmUQ)+1)]    

eeUQ = df.Event1.unique()
eeNO=[]
[eeNO.append(x) for x in range(1,len(eeUQ)+1)]
    
    
pUQ = df.Name.unique()
pNO=[]
[pNO.append(x) for x in range(1,len(pUQ)+1)]
               
df['Event1'].replace(eeUQ,eeNO,inplace=True)
df['Event2'].replace(eeUQ,eeNO,inplace=True)
df['Domain'].replace(dmUQ,dmNO,inplace=True)
df['Name'].replace(pUQ,pNO,inplace=True)
    
tmp_df=df.copy()

In [6]:
inst = open("Domain_extract_file.pkl","rb")
Domain_extract = pickle.load(inst)
inst2 = open("Event_extract_file.pkl","rb")
Event_extract = pickle.load(inst2)

In [8]:
c=0
event_list=[]

while(c==0):
    try:
        event_no = int(input('Enter number of events: '))
        c=1
    except:
        print('ENTER A NUMBER!')
        
for x in range(1,event_no+1):
    event = (input(f'Enter Event No.{x} name: ')).upper()
    event_list.append(event)
    print(f'>>>{event}')

Enter number of events: 1
Enter Event No.1 name: blockchain seminar
>>>BLOCKCHAIN SEMINAR


In [9]:
def dom_eve_extractor(t):
    flag_domain = 0
    words = t.split()
    import re
    import string
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    words = [re_punc.sub('', x) for x in words]
    DE= []
    DE_ml= []
    for x in words:
        try:
            DE.append(Domain_extract[x])
            flag_domain = 1
        except:
            pass
    
    if flag_domain == 0:
        for x in words:
            for y in words:
                try:
                    dummy = x+' '+y
                    DE.append(Domain_extract[dummy])
                except:
                    pass   
                
    f = open('dict_nlp.pkl','rb')
    dict_nlp =pickle.load(f)
    f.close()
    nlp_df = DataFrame(list(dict_nlp.items()),columns = ['Domain','txt']) 
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    
    from sklearn import svm
    clf_svm = svm.SVC(kernel='rbf')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
    
    clf_svm = svm.SVC(kernel='linear')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
    
    from sklearn.tree import DecisionTreeClassifier
    clf_tree = DecisionTreeClassifier()
    clf_tree.fit(train_vectors,nlp_df.Domain)
    pred = clf_tree.predict(temp1)[0]
    DE_ml.append(pred)
    
    from sklearn.naive_bayes import GaussianNB
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
    pred = clf_bay.predict(temp1.toarray())[0]
    DE_ml.append(pred)
    
    
    from sklearn.ensemble import RandomForestClassifier
    clf_RF = RandomForestClassifier()
    clf_RF.fit(train_vectors,nlp_df.Domain)
    pred = clf_RF.predict(temp1)[0]
    DE_ml.append(pred)
    
    
    for x in DE:
        if x not in DE_ml:
            DE_ml.append(x)
    DE_ml = list(dict.fromkeys(DE_ml))
    
    for x in DE_ml:
        if x not in DE:
            io=1
            DE_ml=DE
    
    if (len(DE_ml) == 0):
        DE_ml.append('Other') 
        
    print(DE_ml)
    return DE_ml


In [10]:
def eve_ext(t):
    t=t.split()
    import re
    import string
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    t = [re_punc.sub('', x) for x in t]
    EE= []
    for x in t:
        try:            
            EE.append(Event_extract[x])
        except:
            pass
    print(EE)
    return EE

## Training with Event 1

In [11]:
def traineventone(tmp_df,printscoreone):
    x_train = tmp_df[['Event1','Domain']]
    y_train = tmp_df['Name']
    req_ENO =[]
    req_DNO =[]
    clf = tree.DecisionTreeClassifier()

    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_train)
    

    def p1():
        for x,y in zip(eeNO,eeUQ):
            if y in EE:
                req_ENO.append(x)  
        for x,y in zip(dmNO,dmUQ):
            if y in DE:
                req_DNO.append(x) 
                
        def p2(req_ENO,req_DNO):
            flagg=0
            if(len(req_ENO)>len(req_DNO)):
                diff=len(req_ENO)-len(req_DNO)    
                [req_DNO.append(req_DNO[-1]) for i in range(0,diff)]
                
            else:
                diff=len(req_DNO)-len(req_ENO)
                try:
                    [req_ENO.append(req_ENO[-1]) for i in range(0,diff)]
                    flagg=1
                except:
                    pass
                
            if flagg==1:
                for e,d in zip(req_ENO,req_DNO):
                    x_new=[[e,d]]   
                    tmp = clf.predict_proba(x_new)
                    for index, prob in enumerate(tmp[0]):
                        if prob>=0.5:
                            for x,y in zip(pNO,pUQ):
                                if x==index+1:
                                    temp_name_list.append(y)
        p2(req_ENO,req_DNO)
    p1()

## With Event 2

In [12]:
def traineventtwo(tmp_df,printscoretwo):
    x_train = df[['Event2','Domain']]
    y_train = df['Name']
    req_ENO =[]
    req_DNO =[]

    clf = tree.DecisionTreeClassifier()
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict(x_train)
    def d1():
        for x,y in zip(eeNO,eeUQ):
            if y in EE:
                req_ENO.append(x)
        for x,y in zip(dmNO,dmUQ):
            if y in DE:
                req_DNO.append(x)
                
        def d2(req_ENO,req_DNO):
            flagg=0
            if(len(req_ENO)>len(req_DNO)):
                diff=len(req_ENO)-len(req_DNO)
                [req_DNO.append(req_DNO[-1]) for i in range(0,diff)]

            else:
                diff=len(req_DNO)-len(req_ENO)
                try:
                    [req_ENO.append(req_ENO[-1]) for i in range(0,diff)]
                    flagg=1
                except:
                    pass
            
            if flagg==1:
                for e,d in zip(req_ENO,req_DNO):
                    x_new=[[e,d]]
                    yprob = clf.predict_proba(x_new)
                    y_pred = clf.predict(x_new)
                    for indx,prob in enumerate(yprob[0]):
                        if prob>=0.5:
                            for x,y in zip(pNO,pUQ):
                                if x==indx+1:
                                    temp_name_list.append(y)
        d2(req_ENO,req_DNO)
    d1()

In [13]:
words2num_flag = 0

for t in event_list:
    DE = dom_eve_extractor(t)
    EE = eve_ext(t)
    traineventone(tmp_df,printscoreone)
    traineventtwo(tmp_df,printscoretwo)
    name_list.append(temp_name_list)
    temp_name_list=[]

['Blockchain']
['Seminars']


In [14]:
dict1 = {}
dict2 = {}

In [15]:
for totalppl,event in zip(name_list,event_list):
    for person in totalppl:
        if person not in dict1:
            dict1[person]=event
        elif person in dict1:
            dict2[person]=event

In [16]:
dfx = DataFrame(list(dict1.items()),columns = ['Employee Name','Events']) 

In [17]:
if bool(dict2):
    dfy = DataFrame(list(dict2.items()),columns = ['Employee Name','Events'])
    dfx = dfx.append(dfy).reset_index()
    dfx.drop(columns='index')
dfx

,Employee Name,Events
0,James Porter,BLOCKCHAIN SEMINAR
1,William Turner,BLOCKCHAIN SEMINAR


In [20]:
dfx.to_excel('CloudEvents.xlsx', index = None, header=True)